First analysis of the Hainich data using GPFA for filling the gaps

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from gpfa_imputation.imputation import *

import pandas as pd
from pathlib import Path
from pyprojroot import here

## Load Data

take the first 200 rows from the Hainich dataset

In [3]:
hai_path = Path("FLX_DE-Hai_FLUXNET2015_FULLSET_HH_2000-2012_1-4.csv")
hai_raw = pd.read_csv(here("data") / hai_path, na_values=["-9999", "-9999.99"], parse_dates=[0, 1], nrows=200)

In [4]:
meteo_vars = {
    "TA_F": "TA",
    "SW_IN_F": "SW_IN",
    "LW_IN_F": "LW_IN",
    "VPD_F": "VPD",
    #"PA": "PA"
}

hai = (hai_raw
       .rename(columns=meteo_vars)
       .set_index("TIMESTAMP_END")
       .loc[:, meteo_vars.values()])
hai

,TA,SW_IN,LW_IN,VPD
TIMESTAMP_END,,,,
2000-01-01 00:30:00,-0.60,0.0,302.475,0.222
2000-01-01 01:00:00,-0.65,0.0,302.475,0.122
2000-01-01 01:30:00,-0.58,0.0,301.677,0.090
2000-01-01 02:00:00,-0.51,0.0,301.677,0.110
2000-01-01 02:30:00,-0.49,0.0,301.677,0.102
...,...,...,...,...
2000-01-05 02:00:00,4.74,0.0,330.202,1.191
2000-01-05 02:30:00,4.75,0.0,330.202,1.057
2000-01-05 03:00:00,4.76,0.0,330.202,0.935


In [5]:
gpfa_data = GPFADataTest(hai).add_random_missing()

In [6]:
gpfa_hai = GPFAImputation(gpfa_data.data)

In [7]:
gpfa_hai

GPFA Imputation:
    N obs: 200
    N features 4 (TA, SW_IN, LW_IN, VPD)
    N missing observations 161

In [8]:
imputed = gpfa_hai.impute()

/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/gpytorch/lazy/triangular_lazy_tensor.py:136: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2183.)
  res = torch.triangular_solve(right_tensor, self.evaluate(), upper=self.upper).solution
/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/torch/autograd/__init__.py:173: UserWarning: operator() profile_node %840 : int[] = prim::profile_ivalue(%838)
 does not have profile information (Triggered internally at  ../torch/csrc/jit/codegen/cuda/graph_fuser.cpp:104.)
  Variable._execution_engine.run_backward(  # Calls into the C++ eng

In [9]:
    imputed

,time,variable,mean,std
0,1.0,TA,-0.650000,NaN
1,2.0,TA,-0.580000,NaN
2,4.0,TA,-0.490000,NaN
3,7.0,TA,-0.350000,NaN
4,9.0,TA,-0.270000,NaN
...,...,...,...,...
359,187.0,VPD,0.791283,0.249033
360,189.0,VPD,0.798033,0.249007
361,191.0,VPD,0.797612,0.248991
362,193.0,VPD,0.797653,0.248996


In [10]:
gpfa_hai.plot_pred(gpfa_data.tidy_df(complete=True, is_missing=True))

/home/simone/anaconda3/envs/data-science/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [11]:
#gpfa_hai.plot_pred(complete= gpfa_data.tidy_df(complete=True, is_missing=True) )